In [1]:
import codecs
import re
import requests
from bs4 import BeautifulSoup

# URLとキーワードリスト
url = "https://ja.wikipedia.org/wiki/"
keyword_list = [
    "中島敦", "太宰治", "国木田独歩",
    "江戸川乱歩", "谷崎潤一郎", "宮沢賢治",
    "与謝野晶子", "芥川龍之介", "樋口一葉",
    "中原中也", "尾崎紅葉", "梶井基次郎",
    "夢野久作", "森鷗外", "織田作之助"   
]

# Wikipediaの小説家の記事のダウンロード
corpus = []
for keyword in keyword_list:
    response = requests.get(url + keyword)
   
    soup = BeautifulSoup(response.text, "html.parser")
    for p_tag in soup.find_all('p'):
        # 空白の削除
        text = "".join(p_tag.text.strip().split(" "))

        # 空行は無処理
        if len(text) == 0:
            continue

        # 注釈の削除 (例: [注釈1], [注釈1], [1])
        text = re.sub(r"\[注釈[0-9]+\]", "", text)
        text = re.sub(r"\[注[0-9]+\]", "", text)
        text = re.sub(r"\[[0-9]+\]", "", text)

        # 行の追加
        corpus.append(text)

# ファイルの保存       
print(*corpus, sep="\n", file=codecs.open("wiki.txt", "w", "utf-8"))
import sentencepiece as spm

# 学習の実行
spm.SentencePieceTrainer.Train(
   '--input=wiki.txt --model_prefix=sentencepiece --vocab_size=8000 --character_coverage=0.9995'
)

In [2]:
import sentencepiece as spm

# モデルの作成
sp = spm.SentencePieceProcessor()
sp.Load("sentencepiece.model")
# テキストを語彙列に分割
print(corpus[0])
print(sp.EncodeAsPieces(corpus[0]))

中島敦（なかじまあつし、1909年（明治42年）5月5日-1942年（昭和17年）12月4日）は、日本の小説家。代表作は『山月記』『光と風と夢』『弟子』『李陵』など。第一高等学校、東京帝国大学を卒業後、横浜高等女学校の教員勤務のかたわら小説執筆を続け、パラオ南洋庁の官吏（教科書編修書記）を経て専業作家になるも、同年中に持病の喘息悪化のため33歳で病没。死後に出版された全集は毎日出版文化賞を受賞した。
['▁', '中島敦', '(', 'なか', 'じ', 'ま', 'あ', 'つ', 'し', '、190', '9', '年', '(', '明治', '4', '2', '年', ')', '5', '月', '5', '日', '-194', '2', '年', '(', '昭和', '17', '年', ')', '12', '月', '4', '日', ')', 'は', '、', '日本の小説家', '。', '代表作', 'は', '『', '山月記', '』『', '光と風と夢', '』『', '弟子', '』『', '李陵', '』', 'など', '。', '第一高等学校', '、', '東京帝国大学', 'を', '卒業後', '、', '横浜高等女学校', 'の', '教員', '勤務', 'の', 'かたわら', '小説', '執筆', 'を続け', '、', 'パラオ南洋庁', 'の', '官', '吏', '(', '教科書', '編修書記', ')', 'を経て', '専業作家', 'になる', 'も', '、', '同年', '中', 'に持病の', '喘息悪化', 'のため', '33', '歳で病没', '。', '死後に', '出版された', '全集', 'は', '毎日出版文化賞を受賞し', 'た', '。']


In [6]:
# テキストを語彙IDに分割
print(corpus[0])
print(sp.EncodeAsIds(corpus[0]))
print(sp.encode_as_ids("python前処理を実行する"))

中島敦（なかじまあつし、1909年（明治42年）5月5日-1942年（昭和17年）12月4日）は、日本の小説家。代表作は『山月記』『光と風と夢』『弟子』『李陵』など。第一高等学校、東京帝国大学を卒業後、横浜高等女学校の教員勤務のかたわら小説執筆を続け、パラオ南洋庁の官吏（教科書編修書記）を経て専業作家になるも、同年中に持病の喘息悪化のため33歳で病没。死後に出版された全集は毎日出版文化賞を受賞した。
[20, 358, 11, 1013, 190, 269, 230, 116, 27, 754, 51, 9, 11, 50, 35, 30, 9, 12, 40, 19, 40, 26, 2631, 30, 9, 11, 54, 200, 9, 12, 72, 19, 35, 26, 12, 10, 3, 979, 5, 983, 10, 15, 417, 109, 720, 109, 480, 109, 723, 14, 39, 5, 1479, 3, 2479, 7, 1294, 3, 4230, 4, 2298, 690, 4, 1953, 80, 484, 2231, 3, 6640, 4, 280, 7801, 11, 1036, 6086, 12, 1107, 6354, 680, 22, 3, 232, 74, 6427, 6459, 284, 1312, 5735, 5, 3384, 3401, 328, 10, 5060, 24, 5]
[20, 845, 2964, 2930, 5169, 776, 1890, 141, 6861, 7, 296, 178, 37]


In [15]:
from pyknp import Juman
jumanpp = Juman()
tokens = jumanpp.analysis(text)
for mrph in tokens.mrph_list():
    print(mrph.midasi)

Exception: Can't find JUMAN command: jumanpp